In [ ]:
import pandas as pd
from sklearn.metrics.cluster import v_measure_score
from sklearn.metrics.cluster import normalized_mutual_info_score
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import os
import sys
sys.path.append('/home/stankeaa/wikiwho_inheritance/')
import re
from sequencers.clusterers import kmeans, dbscan
%matplotlib inline  
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn.cluster import KMeans

In [ ]:
def getting_vectorizers(dirs):
    dir_to_vecs = {}
    for folder in dirs:
        dir_to_vecs[folder] = os.listdir(sys.path[-1] + folder + '/clusterers/')[1:]
    return dir_to_vecs

def getting_pickles(vecs):
    vecs_to_clusters = {}
    for vectorizer in vecs:
        path = sys.path[-1] + folder + '/clusterers/' + vectorizer + '/tsne/'
        pickles = []
        for dirpath, dirnames, files in os.walk(path):
            for file_name in files:
                pickles.append(file_name)
        vecs_to_clusters[vectorizer] = pickles        
    return vecs_to_clusters

dirs = []
for file in os.listdir( sys.path[-1] ):
    if re.match("^[0-9_-]*$", file):
        dirs.append(file) # getting the chobs
dir_to_vecs = getting_vectorizers(dirs) # getting the vectorizers
for folder in dir_to_vecs:
    vecs = dir_to_vecs[folder]
    vecs_to_clusters = getting_pickles(vecs) # getting the clusterers (with tsne)

## V-scores by vectorizer, comparing the clusterizers

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5
for vec in vecs_to_clusters:
    v_measure_scores = {}
    for clu in vecs_to_clusters[vec]:
        df = pd.read_pickle('../39570/clusterers/' + vec + '/tsne/' + clu)
        nationality = df.loc[~df['nationality'].isna()]
        score = v_measure_score(nationality['nationality'], nationality['clusters'])
        if clu.split('_')[0] == 'Token':
            if 'id' in clu:
                clu = 'Token_Sim_ID'
            else:
                clu = 'Token_Sim_Str'
        else:
            clu = clu.split('_')[0]
        if clu in v_measure_scores.keys() and score > v_measure_scores[clu]: 
                v_measure_scores[clu] = score
        elif clu not in v_measure_scores.keys():
            v_measure_scores[clu] = score
        
    plt.bar(range(len(v_measure_scores)), list(v_measure_scores.values()), align='center')       
    plt.xticks(range(len(v_measure_scores)), list(v_measure_scores.keys()))
    print(vec)
    plt.show()

## V-scores by clasterizer, comparing the parameters

In [ ]:
def print_data(vectorizer):
    if vectorizer is not None:
        print('Downloaded data for {} vectorizer'.format(vectorizer)) 
    global vec, df
    vec = vectorizer
    #df = pd.read_pickle(sys.path[-1] + d.value + '/clusterers/' + vec + '/tsne/' + clu)
    
def select_vec(folder):
    v.options = dir_to_vecs[folder]

#add in 'select clusterizer' function that looks in the new dictionary
def select_clusterizer(vectorizer):
    c.options = vecs_to_clusters[vectorizer]

d = widgets.Dropdown(options=dir_to_vecs.keys())
init = d.value
v = widgets.Dropdown(options=dir_to_vecs[init])


init2= v.value #new start value for vectorizer dropdown

j = widgets.interactive(print_data, vectorizer=v) #define clusterizer value
i = widgets.interactive(select_vec, folder=d)

display(i)
display(j)

In [ ]:
v_measure_scores_db = {}
v_measure_scores_ts = {}
#global_min = 100
for clu in vecs_to_clusters[vec]:
    if clu.split('_')[0] == 'DBscan':
        eps = clu.split('_')[5][:-4]
        min_samples = clu.split('_')[3]
        if min_samples not in v_measure_scores_db.keys():
            v_measure_scores_db[min_samples] = []
        df = pd.read_pickle('../39570/clusterers/' + vec + '/tsne/' + clu)
        nationality = df.loc[~df['nationality'].isna()]
        score = v_measure_score(nationality['nationality'], nationality['clusters'])
        #if int(min_samples) < global_min:
            #global_min = int(min_samples)
        v_measure_scores_db[min_samples].append((score, float(eps)))
    elif clu.split('_')[0] == 'Token':
        if 'id' in clu:
            intersect = clu.split('_')[3]
            df = pd.read_pickle('../39570/clusterers/' + vec + '/tsne/' + clu)
            nationality = df.loc[~df['nationality'].isna()]
            score = v_measure_score(nationality['nationality'], nationality['clusters'])
            v_measure_scores_ts[int(intersect)] = score

In [ ]:
'''Running k-means with different k to select an optimal, takes time'''
def transform_feat(df):
    features = pd.DataFrame()
    for i, row in df.iterrows():
        feat = pd.Series(row['features'])
        features = features.append(feat,ignore_index=True)
    return features

v_measure_scores_km = {}
K = range(4,15)
for k in K:
    km = KMeans(n_clusters=k)
    km.fit(transform_feat(df))
    df['clusters'] = km.labels_
    nationality = df.loc[~df['nationality'].isna()]
    score = v_measure_score(nationality['nationality'], nationality['clusters'])
    v_measure_scores_km[k] = score

In [ ]:
print('Clusterizers for ' + vec)
if len(v_measure_scores_db) > 0:
    plt.figure()
    for min_sample in v_measure_scores_db.keys():
        v_measure_scores_sort = sorted(v_measure_scores_db[min_sample], key=lambda tup: tup[1])
        x_val = [x[1] for x in v_measure_scores_sort]
        y_val = [x[0] for x in v_measure_scores_sort]
        plt.plot(x_val, y_val, label='Min sample ' + min_sample)

    plt.legend(loc='upper right')
    plt.title("DBscan")
    plt.xlabel('Eps')
    plt.ylabel('V-measure score')
    plt.show()

if len(v_measure_scores_ts) > 0:
    plt.figure()
    ts = sorted(v_measure_scores_ts.items()) 
    x, y = zip(*ts)
    plt.plot(x, y)
    plt.title("Token Similarity ID")
    plt.xlabel('Intersection')
    plt.ylabel('V-measure score')
    #plt.figure(figsize=(8, 6))
    plt.show()
    
if len(v_measure_scores_km) > 0:
    plt.figure()
    km = sorted(v_measure_scores_km.items()) 
    x, y = zip(*km)
    plt.plot(x, y)
    plt.title("Kmeans")
    plt.xlabel('number of clusters')
    plt.ylabel('V-measure score')
    #plt.figure(figsize=(8, 6))
    plt.show()

## Score for one pickle

In [ ]:
def print_data(vectorizer,clusterizer):
    if clusterizer is not None:
        print('Downloaded data for {} vectorizer and {} clusterizer'.format(vectorizer, clusterizer[:-4])) 
    global vec, clu, df
    vec, clu = vectorizer, clusterizer
    df = pd.read_pickle(sys.path[-1] + d.value + '/clusterers/' + vec + '/tsne/' + clu)
    
def select_vec(folder):
    v.options = dir_to_vecs[folder]

#add in 'select clusterizer' function that looks in the new dictionary
def select_clusterizer(vectorizer):
    c.options = vecs_to_clusters[vectorizer]

d = widgets.Dropdown(options=dir_to_vecs.keys())
init = d.value
v = widgets.Dropdown(options=dir_to_vecs[init])


init2= v.value #new start value for vectorizer dropdown
c = widgets.Dropdown(options=vecs_to_clusters[init2]) #define district dropdown widget

j = widgets.interactive(print_data, vectorizer=v, clusterizer=c) #define clusterizer value
i = widgets.interactive(select_vec, folder=d)
k = widgets.interactive(select_clusterizer, vectorizer=v) #call everything together with new interactive

display(i)
display(j)

In [ ]:
nationality = df.loc[~df['nationality'].isna()]
v_score = v_measure_score(nationality['nationality'], nationality['clusters'])
n_score = normalized_mutual_info_score(nationality['nationality'], nationality['clusters'])
print('For vectorizer ', vec, 'and clusterizer ', clu, '\nthe v-measure score is: ', v_score, '\nthe normalized mutual info score is: ', n_score)

## Entropy, general for one pickle

In [ ]:
from scipy.stats import entropy
nationality = df.loc[~df['nationality'].isna()]

def get_weighted_entropy(dataframe, entropy_column, group_columns="cluster"):
    group_size = dataframe.groupby(group_columns).size()
    group_entropy = dataframe.groupby(group_columns)[entropy_column].apply(lambda x: entropy(x.value_counts().values))
    absolute_entropy = group_size * group_entropy
    weighted_entropy = (group_size * group_entropy).mean()
    return group_entropy, weighted_entropy, group_size

group_entropy, weighted_entropy, group_size = get_weighted_entropy(nationality, group_columns="clusters", entropy_column="nationality")

In [ ]:
print('General entropy for ' + vec + ' ' + clu[:-4])
entropy_df = pd.DataFrame(group_entropy).rename(columns={"nationality": "entropy"})
entropy_df = entropy_df.merge(pd.DataFrame(data = group_size), left_index=True, right_index=True).rename(columns={0: "size"})
entropy_df = entropy_df.sort_values(by='entropy', ascending=False)
entropy_df.index  = entropy_df.index.map(str)
entropy_df

In [ ]:
width = 0.7
entropy_df['entropy'].plot(kind='bar', width = width)
plt.ylabel('Entropy')
plt.title('Entropy chart for ' + vec + ' ' + clu[:-4])
entropy_df['size'].plot(secondary_y=True, color='r')
plt.ylabel('Cluster size')
plt.legend()
plt.xlim([-width, len(entropy_df['entropy'])-width])
plt.show()

## General entropy, for one clusterizers. Comparing the parameters

In [ ]:
entropy_db = {}
entropy_ts = {}
for clu in vecs_to_clusters[vec]:
    if clu.split('_')[0] == 'DBscan':
        eps = clu.split('_')[5][:-4]
        min_samples = clu.split('_')[3]
        if min_samples not in entropy_db.keys():
            entropy_db[min_samples] = []
        df = pd.read_pickle('../39570/clusterers/' + vec + '/tsne/' + clu)
        nationality = df.loc[~df['nationality'].isna()]
        entropy_val, weighted_entropy, group_size = get_weighted_entropy(nationality, group_columns="clusters", entropy_column="nationality")
        entropy_db[min_samples].append((weighted_entropy, float(eps)))
    elif clu.split('_')[0] == 'Token':
        if 'id' in clu:
            intersect = clu.split('_')[3]
            df = pd.read_pickle('../39570/clusterers/' + vec + '/tsne/' + clu)
            nationality = df.loc[~df['nationality'].isna()]
            entropy_val, weighted_entropy, group_size = get_weighted_entropy(nationality, group_columns="clusters", entropy_column="nationality")
            entropy_ts[int(intersect)] = weighted_entropy

In [ ]:
print('General entropy for clusterizers for ' + vec)
if len(entropy_db) > 0:
    plt.figure()
    for min_sample in entropy_db.keys():
        entropy_sort = sorted(entropy_db[min_sample], key=lambda tup: tup[1])
        x_val = [x[1] for x in entropy_sort]
        y_val = [x[0] for x in entropy_sort]
        plt.plot(x_val, y_val, label='Min sample ' + min_sample)

    plt.legend(loc='upper left')
    plt.title("DBscan")
    plt.xlabel('Eps')
    plt.ylabel('Entropy')
    plt.show()

if len(entropy_ts) > 0:
    plt.figure()
    ts = sorted(entropy_ts.items()) 
    x, y = zip(*ts)
    plt.plot(x, y)
    plt.title("Token Similarity ID")
    plt.xlabel('Intersection')
    plt.ylabel('Entropy')
    #plt.figure(figsize=(8, 6))
    plt.show()

## Position entropy

In [ ]:

nationality = df.loc[~df['nationality'].isna()] #taking only nationality cases
cluster_sizes = nationality.groupby("clusters").size().values
cluster_sizes = cluster_sizes / cluster_sizes.sum()

nationality['relative_position'] = nationality.loc[: , ['left_neigh', 'right_neigh']].mean(axis=1)
position_entropy_clusters = nationality.groupby("clusters")["relative_position"].apply(
    lambda x: entropy(x.value_counts().values))
position_entropy_clusters.plot(kind='bar')
plt.ylabel('Position entropy')
plt.title('Position entropy chart for ' + vec + ' ' + clu[:-4] + ' based on mean position')
position_entropy = (cluster_sizes * position_entropy_clusters).sum()
avg_position_entropy =  position_entropy_clusters.mean()
position_entropy

## Token entropy

In [ ]:

token_entropy_clusters = nationality.groupby("clusters")["ins_tokens_str"].apply(
    lambda token_tuples: entropy(pd.Series(
        [token for token_tuple in token_tuples.tolist() for token in token_tuple]
    ).value_counts().values))

token_entropy_clusters.plot(kind='bar')
plt.ylabel('Token entropy')
plt.title('Token entropy chart for ' + vec + ' ' + clu[:-4] + ' based on inserted string tokens')
position_entropy = (cluster_sizes * position_entropy_clusters).sum()
token_entropy = (cluster_sizes * token_entropy_clusters).sum()
avg_token_entropy =  token_entropy_clusters.mean()
token_entropy

## Comparing position and token entropies for different parameters

In [ ]:
entropy_pos = {}
entropy_tok = {}
for clu in vecs_to_clusters[vec]:
    if clu.split('_')[0] == 'DBscan':
        eps = clu.split('_')[5][:-4]
        min_samples = clu.split('_')[3]
        if min_samples not in entropy_pos.keys(): #adding new min_sample as a key to the dict
            entropy_pos[min_samples] = []
        if min_samples not in entropy_tok.keys(): #adding new min_sample as a key to the dict
            entropy_tok[min_samples] = []
        df = pd.read_pickle('../39570/clusterers/' + vec + '/tsne/' + clu)
        #nationality = df.loc[~df['nationality'].isna()] #taking only nationality cases
        df['relative_position'] = df.loc[: , ['left_neigh', 'right_neigh']].mean(axis=1) # calculating relative position as a mean between the beggining and the end of the context
        cluster_sizes = df.groupby("clusters").size().values
        cluster_sizes = cluster_sizes / cluster_sizes.sum()
        position_entropy_clusters = df.groupby("clusters")["relative_position"].apply(
            lambda x: entropy(x.value_counts().values)) #calculating the entropy itself
        position_entropy = (cluster_sizes * position_entropy_clusters).sum() #calculating the total weighted position entropy
        entropy_pos[min_samples].append((position_entropy, float(eps))) #appending a value to the list for respective min_sample
        
        token_entropy_clusters = df.groupby("clusters")["ins_tokens_str"].apply(
        lambda token_tuples: entropy(pd.Series(
            [token for token_tuple in token_tuples.tolist() for token in token_tuple]
        ).value_counts().values))
        token_entropy = (cluster_sizes * token_entropy_clusters).sum() # calculating the total weighted token entropy
        entropy_tok[min_samples].append((token_entropy, float(eps))) #appending a value to the list for respective min_sample

In [ ]:
print('Position and token entropy for clusterizers for ' + vec)
if len(entropy_pos) > 0:
    plt.figure()
    for min_sample in entropy_pos.keys():
        entropy_sort = sorted(entropy_pos[min_sample], key=lambda tup: tup[1])
        x_val = [x[1] for x in entropy_sort]
        y_val = [x[0] for x in entropy_sort]
        plt.plot(x_val, y_val, label='Min sample ' + min_sample)

    plt.legend(loc='upper left')
    plt.title("Position entropy")
    plt.xlabel('Eps')
    plt.ylabel('Entropy')
    plt.show()

if len(entropy_tok) > 0:
    plt.figure()
    for min_sample in entropy_tok.keys():
        entropy_sort = sorted(entropy_tok[min_sample], key=lambda tup: tup[1])
        x_val = [x[1] for x in entropy_sort]
        y_val = [x[0] for x in entropy_sort]
        plt.plot(x_val, y_val, label='Min sample ' + min_sample)

    plt.legend(loc='upper left')
    plt.title("Token entropy")
    plt.xlabel('Eps')
    plt.ylabel('Entropy')
    plt.show()

## Silhouette score

In [ ]:
#getting the average score and silhouette cluster scores from the df
sil_avg = df.loc[0, 'avg_sil']
sil_df = df.copy()
if clu.split('_')[0] == 'Kmeans':
    cl = sil_df.clusters.unique()
elif clu.split('_')[0] == 'DBscan':    
    cl = sil_df.clusters.value_counts()[1:11].index
    
    
plt.figure(figsize=(18, 7))
plt.xlim(-1, 1)
print("For n_clusters =", len(cl),
      "The average silhouette_score is :", sil_avg)
y_lower = 10
for i in cl:
    # Aggregate the silhouette scores for samples belonging to
    # cluster i, and sort them
    ith_cluster_silhouette_values = \
        sil_df[sil_df["clusters"]==i]["silhouette_value"]

    ith_cluster_silhouette_values.sort_values()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    plt.fill_betweenx(np.arange(y_lower, y_upper),
                      0, ith_cluster_silhouette_values,
                       alpha=0.7)

    # Label the silhouette plots with their cluster numbers at the middle
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Compute the new y_lower for next plot
    y_lower = y_upper + 10  # 10 for the 0 samples

plt.title("The silhouette plot for the various clusters.")
plt.xlabel("The silhouette coefficient values")
plt.ylabel("Cluster label")

# The vertical line for average silhouette score of all the values
plt.axvline(x=sil_avg, color="red", linestyle="--")

plt.yticks([])  # Clear the yaxis labels / ticks
plt.xticks([-1, -0.6,-0.4,-0.2,-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
plt.show()
#plt.savefig(fname+'_silhouette.png')

In [ ]:
from sklearn.neighbors import NearestNeighbors
def transform_feat(df):
        features = pd.DataFrame()
        for i, row in df.iterrows():
            feat = pd.Series(row['features'])
            features = features.append(feat,ignore_index=True)
        return features

ns = 5
nbrs = NearestNeighbors(n_neighbors=ns).fit(transform_feat(df))
distances, indices = nbrs.kneighbors(transform_feat(df))
distanceDec = sorted(distances[:,ns-1])#, reverse=True)

In [ ]:
plt.plot(list(range(1,len(df)+1)), distanceDec)